In [1]:
import numpy as np
import pandas as pd
import re
import nltk
from mediawiki import MediaWiki
from bs4 import BeautifulSoup
import string
from nltk import tokenize
from nltk.tag import StanfordNERTagger
import string
from nltk.tokenize import word_tokenize
import unicodedata
import timeit

In [2]:
# Using the Stanfors' default tagger
st = StanfordNERTagger('/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/classifiers/english.all.3class.distsim.crf.ser.gz',
  '/Users/venkatakrishnamohansunkara/Desktop/DM/stanford-ner-2018-02-27/stanford-ner.jar',
   encoding='utf-8') 

In [3]:
# Lists containing common prefixes and suffixes.
prefix = ['at','by','near','in','from', 'on','to']
# https://www.irfca.org/docs/place-names.html
suffixes = ['nagar','colony','street','road','hill','river','temple']
subs = ['pur','puram','ur','ghar','pura','ganj','abad','halli','keri']

In [4]:
# Read the data
#news_data = pd.read_csv('/Users/venkatakrishnamohansunkara/Desktop/DM/GDELT_2017/large_dataset/unrest_2017_ie_toi.csv', sep=',')
df = pd.read_csv('../document similarity/document_similarity_hatt.csv')

In [5]:
df

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,category,content,doc2vec,doc2veccosine,HATT embeddings
0,0,0,0,movie,Padmaavat: Why a Bollywood epic has sparked fi...,[ -9.05430984 1.71762216 1.08929253 -3.22...,[ 1. 0.94052142 0.9796434 0.926054...,[-0.04064541 0.5168978 -0.10278964 -0.406168...
1,1,1,1,lokpal,Anna Hazare on fast-unto-death demanding Jan L...,[ -4.08943748e+00 3.65195364e-01 4.3100047...,[ 0.94052142 1. 0.97000271 0.940714...,[ 1.61063448e-01 5.65693378e-01 -4.65846695e-...
2,2,2,2,movie,A Film Has Inflamed Indians. But Moviegoers Mo...,[ -8.49335098 0.55525446 1.04886711 -3.75...,[ 0.9796434 0.97000271 1. 0.946262...,[ 1.81895524e-01 5.93335807e-01 6.25734329e-...
3,3,3,3,movie,Indian police detain dozens protesting against...,[ -6.10597134 0.87642241 0.60653818 -3.09...,[ 0.92605472 0.9407143 0.94626272 0.999999...,[ 0.17177399 0.6309512 -0.04849247 -0.051712...
4,4,4,4,lokpal,Thousands Back Antigraft Hunger Strike in New ...,[-11.33139229 2.65446663 1.2543447 -0.94...,[ 0.8824504 0.86918712 0.90175384 0.953397...,[-0.06123626 -1.1703769 0.01185594 -0.637607...
5,5,5,5,lokpal,Jan Lokpal Bill: Protests continue in support ...,[ -4.41828299e+00 5.35881042e-01 6.4218103...,[ 0.93510097 0.97717178 0.95960432 0.905399...,[-3.85057926e-02 3.16520661e-01 -2.69440532e-...
6,6,6,6,lokpal,India corruption: Protests swell in support of...,[ -7.38126898 0.73874789 1.25690413 -4.65...,[ 0.97860324 0.96208191 0.97947335 0.944147...,[ 1.35564059e-01 5.99828005e-01 -5.64954542e-...
7,7,7,7,lokpal,Hunger strike over Lokpal Bill as thousands pr...,[-5.09889364 0.64079404 0.13031445 -4.445450...,[ 0.95688981 0.98412955 0.96912926 0.928257...,[ 1.44097835e-01 4.18904036e-01 -1.80478878e-...
8,8,8,8,movie,Padmaavat: Riots break out in India as protest...,[ -2.40009403e+00 6.57581329e-01 4.1137728...,[ 0.88772088 0.95705724 0.91422349 0.880223...,[ 0.03712982 0.63247323 -0.16278782 -0.318585...


In [6]:
test_data = df.iloc[5].content

In [7]:
test_data

'Jan Lokpal Bill: Protests continue in support of Anna Hazare in Punjab\n\nCHANDIGARH: Khap panchayats today joined the protests in support of Anna Hazare in Punjab and Haryana where the Gandhian\'s supporters took out car and bike rallies and staged a demonstration outside Prime Minister Manmohan Singh\'s private residence here.\xa0\n\nAs the protests spread, people in rural areas were seen participating in demonstrations in support of Hazare.\xa0\n\nKhap spokesman Suresh Koth said the khap panchayats (caste councils) in Haryana are observing a hunger strike in Jind, the Jat heartland of Haryana, and other places in support of Hazare\'s movement.\xa0\n\nIn Chandigarh, activists of various organisations staged a protest outside the Prime Minister\'s private residence in Sector 11.\xa0\n\nThe Chandigarh administration had yesterday promulgated prohibitory orders banning assembly of five or more persons outside the residences of five VIPs, including that of the Prime Minister.\xa0\n\nThe

In [3]:
# Function for removing unnecessary characters.
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", " ", string)
    string = re.sub(r"\'", " ", string)
    string = re.sub(r"\"", " ", string)
    string = re.sub(r"\n", " ", string)
    string = re.sub(r"=", " ", string)
    string = re.sub(r"-", " ", string)
    string = re.sub(r"/", " ", string)
    return string.strip()

In [4]:
sent = "The attacks have raised grave concern both with the society and the government at large."
text_tags = nltk.word_tokenize(sent)
original_tags = st.tag(text_tags)

In [5]:
original_tags

[('The', 'O'),
 ('attacks', 'O'),
 ('have', 'O'),
 ('raised', 'O'),
 ('grave', 'O'),
 ('concern', 'O'),
 ('both', 'O'),
 ('with', 'O'),
 ('the', 'O'),
 ('society', 'O'),
 ('and', 'O'),
 ('the', 'O'),
 ('government', 'O'),
 ('at', 'O'),
 ('large', 'O'),
 ('.', 'O')]

In [9]:
test_data = unicodedata.normalize("NFKD", test_data)
test_data = clean_str(test_data)

In [10]:
start = timeit.default_timer()
# Convert the entire document data to sentences.
text_sent = nltk.sent_tokenize(test_data)
stanford = []
noun_phrases = []
#new_model = []
locations = []
# other are prefixes
other = []
suffix_locations = []
sub_locations = []
all_words = []
# Iterate over all the sentences.
for sent in text_sent:
    # Convert the sentence to words
    text_tags = nltk.word_tokenize(sent)
    all_words+=[text_tags]
    # Get the NER for the words.
    original_tags = st.tag(text_tags)
    l = len(original_tags)
    i=0;
    # Iterate over the tagged words.
    while i<l:
        #print(i)
        e,t = original_tags[i];
        # If it's a location, then check the next 3 words.
        if t == 'LOCATION':
            j = 1;
            s = e; 
            # Verify the tags for the next 3 words.
            while i+j<len(original_tags):
                # If the next words are also locations, then concatenate them to make a large string.
                if original_tags[i+j][1] == 'LOCATION':
                    s = s+" "+original_tags[i+j][0];
                    j+=1;
                else:
                    break;
            i = i+j;
            # Save the locations to a locations list
            locations+=[s];
        else:
            i=i+1;
    # Get the POS tag for the words.
    text_tagged = nltk.pos_tag(text_tags)
    l = len(text_tagged)
    i=0;
    # iterate over the tagged words.
    while i<l:
        #print(i)
        e,t = text_tagged[i];
        # If the current word is a Noun phrase.
        if t == 'NNP':
            j = 1;
            s = e;
            # verify the tags for the next 3 words.
            while i+j<len(text_tagged):
                # If the next words are also nouns, then concatenate them to make a large string.
                if text_tagged[i+j][1] in ['NNP','NN','NNS','NNPS']:
                    s = s+" "+text_tagged[i+j][0];
                    j+=1;
                else:
                    break;
            i = i+j;
            # Save the noun phrases to a noun_phrases list
            noun_phrases+=[s];
        else:
            i=i+1;
    # Convert the words to lower case so as to compare with the prefixes.
    text_tags = [t.lower() for t in text_tags]
    # Go through all the prefixes.
    for k in prefix:
        # If the prefix is not present in the sentence then an error occurs. So, we use a try, catch block
        try:
            # Get the position the prefix word in a sentence.
            index = text_tags.index(k)
            # If the word after the prefix word is a noun, then consider it.
            if text_tagged[index+1][1] == 'NNP':
                j = 1;
                s = text_tagged[index+1][0]; 
                # Verify if it is a phrase by considering the next 3 words.
                while index+j<len(text_tagged):
                    if text_tagged[index+j+1][1] in ['NNP','NN','NNS','NNPS']:
                        s = s+" "+text_tagged[index+j+1][0];
                        j+=1;
                    else:
                        break;
                # Save the words after the prefix words in to other list.
                other = other + [s]
        except:
            continue
    # Iterate through the list of suffixes.
    for s in suffixes:
        try:
            # get the position of the suffix word in the sentence
            index = text_tags.index(s)
            # If the word before a suffix word is a noun phrase, then consider it.
            if text_tagged[index-1][1] == 'NNP':
                j = 1;
                stri = text_tagged[index-1][0]; 
                # verify if the preceeding 3 words are nouns.
                while j<3 and index+j<len(text_tagged):
                    if text_tagged[index-j-1][1] in ['NNP','NN','NNS','NNPS']:
                        stri = stri+" "+text_tagged[index-j-1][0];
                        j+=1;
                    else:
                        break;
                # Reverse the preceeding words list
                stri = ' '.join(stri.split(" ")[-1::-1]) 
                # If the string is not already present in other lists, then add it to the suffix_locations list.
                if stri not in locations or stri not in noun_phrases or stri not in other:
                    stri = stri+" "+ s;
                    if stri not in locations or stri not in noun_phrases or stri not in other:
                        suffix_locations = suffix_locations + [stri]
        except:
            continue
    for sub in subs:
        # get the words which contain any of the sub_string from the subs list.
        su = [te for te in text_tags if te.endswith(sub)]
        try:
            for sub1 in su:
                # Get the position of the word from the sentence.
                index = text_tags.index(sub1)
                # If it is a Noun, then consider it.
                if text_tagged[index][1] in ['NNP','NN','NNS','NNPS']:
                    sub_locations = sub_locations + [text_tagged[index][0]]
        except:
            continue
stop = timeit.default_timer()
print('Time: ', stop - start) 
all_words = [x for sublist in all_words for x in sublist]

Time:  20.79032592900012


In [11]:
all_locations = locations+other+suffix_locations+sub_locations

In [12]:
all_locations

['Punjab CHANDIGARH',
 'Punjab',
 'Haryana',
 'Haryana',
 'Jind',
 'Haryana',
 'Chandigarh',
 'Chandigarh',
 'Haryana',
 'Chandigarh',
 'Sector']

In [13]:
all_locations_nodups = []
for i in all_locations:
    if i.lower() not in [j.lower() for j in all_locations_nodups]:
        all_locations_nodups.append(i);

In [14]:
# 5 different conditions for ranking the data.
# A metric for similarity measures.
locations_list = all_locations_nodups

In [15]:
words_freq = nltk.FreqDist(all_words)
#words_freq = sorted(words_freq.items(), key=lambda x: x[1])
words_freq = dict(words_freq)

In [16]:
all_words = np.asarray(all_words)
# get all positional occurences of a word in the document
pos_occurences = {k:-1 for k in locations_list}

In [17]:
vocab = ["strike",
"unrest",
"masses",
"protest",
"demonstration",
"worker",
"union",
"company",
"caste",
"religious",
"ethnic",
"reformed",
"rebellion",
"defense",
"violence",
"war",
"armed",
"fight",
"Right",
"free",
"freedom",
"liberty",
"justice",
"Fair",
"unfair",
"unequal",
"terror",
"extreme",
"Bomb",
"IED",
"weapon",
"gun",
"wmd",
"threat",
"suicide",
"murder",
"Kill",
"death",
"explosive",
"military",
"police",
"elite",
"government",
"oppresive",
"power",
"regime",
"fraud",
"corruption",
"coup",
"safety",
"secure",
"insecure",
"protect",
"enemy",
"resist",
"hostage",
"truce",
"fire",
"greed",
"panic",
"inflation",
"Price",
"Food",
"Water"]

In [18]:
vocab = [v.lower() for v in vocab]

In [19]:
pos_threshold = len(all_words)
all_locations = {k:[pos_threshold+1,'r',''] for k in locations_list}
for location in list(all_locations.keys()):
    sub_locations = location.split(' ')
    if len(sub_locations) > 1:
        last_subloc = sub_locations[-1]
        first_subloc = sub_locations[0]
        last_indices = np.where(all_words == last_subloc)[0]
        first_indices = np.where(all_words == first_subloc)[0]
        first_indices = list(first_indices)
        last_indices = list(last_indices)
        diff = [a_i - b_i for a_i, b_i in zip(last_indices, first_indices)]
        first_indices = first_indices[diff == len(sub_locations)]
        pos_occurences[location] = first_indices
    else:
        indices = np.where(all_words == location)[0]
        pos_occurences[location] = list(indices)
    print(indices)
    if len(indices)>0:
        no_of_indices = len(indices)
        i=0
        while i< no_of_indices:
            j=0
            while j<pos_threshold and indices[i]+j<len(all_words):
                if all_words[indices[i]+j].lower() in vocab:
                    if j<all_locations[location][0]:
                        all_locations[location] = [j,'r',all_words[indices[i]+j].lower()]
                    break
                j=j+1
            j=0
            while j<pos_threshold and indices[i]-j>=0:
                if all_words[indices[i]-j].lower() in vocab:
                    if j<all_locations[location][0]:
                        all_locations[location] = [j,'l',all_words[indices[i]-j].lower()]
                    break
                j=j+1
            i=i+1

NameError: name 'indices' is not defined

In [ ]:
# The nearest position of an unrest keyword.
all_locations

In [ ]:
# The position of the word in the entire document.
pos_occurences

In [ ]:
locations_freq = {k:0 for k in locations_list}

In [ ]:
for k in locations_list:
    sub_locations = k.split(' ')
    if len(sub_locations) > 1:
        min_count = words_freq[sub_locations[0]]
        for s in sub_locations:
            if words_freq[s]<min_count:
                min_count = words_freq[s];
    else:
        min_count = words_freq[k]
    locations_freq[k] = min_count

In [ ]:
# The number of occurences of a location in the document.
# Some cases were ignored.
locations_freq

In [ ]:
sentence_no = {k:[] for k in locations_list}

In [ ]:
for loc in locations_list:
    sub_locations = loc.split(' ')
    sent_no = 0
    for sent in text_sent:
        text_tags = nltk.word_tokenize(sent)
        i=0
        for s in sub_locations:
            if s in text_tags:
                i+=1
        if i == len(sub_locations):
            sentence_no[loc] = sentence_no[loc]+[sent_no]
        sent_no+=1  

In [ ]:
# The sentence number in which the word is present.
sentence_no

In [ ]:
final_scores = {k:0 for k in locations_list}

In [ ]:
total_no_words = len(all_words)
total_no_sentences = len(text_sent)
for loc in locations_list:
    fs = (locations_freq[loc]/total_no_words) + (1-(pos_occurences[loc]/total_no_words)) \
            + (1-(all_locations[loc][0]/pos_threshold)) + (1-(sentence_no[loc][0]/total_no_sentences))
    final_scores[loc] = fs/4;

In [ ]:
final_scores

In [22]:
all_locs = pd.read_csv('all_locations.csv')

In [25]:
all_locations = all_locations.keys()

AttributeError: 'dict_keys' object has no attribute 'keys'

In [26]:
all_locations = list(all_locations)

In [21]:
locations_list

['Punjab CHANDIGARH', 'Punjab', 'Haryana', 'Jind', 'Chandigarh', 'Sector']

In [28]:
# Remove non english characters, everything in ()
# Sort the dataframe
for i,row in all_locs.iterrows():
    aname = row['Area Name']
    aname = re.sub("[\(\[].*?[\)\]]", "", str(aname))
    all_locs.set_value(i,'Area Name',aname)

In [31]:
re.sub("[\(\[].*?[\)\]]", "", "Bathar (480)")

'Bathar '

In [40]:
all_locs.to_csv('all_locations.csv')

In [41]:
names = all_locs['Area Name']

In [42]:
names = list(names)

In [23]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [35]:
final_df = pd.DataFrame(columns=['Area Name','SUB_DISTRICT NAME','DISTRICT NAME','STATE NAME','final_score'])
i=0;
for name in locations_list:
    # Extract the closest match > 98%.
    # If more than one extract all 
    print(name)
    def get_ratio(row):
        aname = row['Area Name']
        return fuzz.ratio(aname,name)
    found_locs = all_locs[all_locs.apply(get_ratio, axis=1) > 98]
    if found_locs.shape[0]>0:
        for j,row in found_locs.iterrows():
            final_df.set_value(i,'Area Name',row['Area Name'])
            final_df.set_value(i,'SUB_DISTRICT NAME',row['SUB-DISTRICT NAME'])
            final_df.set_value(i,'DISTRICT NAME',row['DISTRICT NAME'])
            final_df.set_value(i,'STATE NAME',row['STATE NAME'])
            #final_df.set_value(i,'final_score',score)
            i+=1
    else:
        final_df.set_value(i,'Area Name',name)
        final_df.set_value(i,'SUB_DISTRICT NAME','None')
        final_df.set_value(i,'DISTRICT NAME','None')
        final_df.set_value(i,'STATE NAME','None')
        #final_df.set_value(i,'final_score',score)
        i+=1

Punjab CHANDIGARH
Punjab
Haryana
Jind
Chandigarh
Sector


In [36]:
# Min distance of the nearest unrest keyword, first word occurence of a location mention,...
# first sentence occurence of a location mention, frequency of a word as the components of the final_score.
final_df.drop_duplicates()

,Area Name,SUB_DISTRICT NAME,DISTRICT NAME,STATE NAME,final_score
0,Punjab CHANDIGARH,None,None,None,NaN
1,Punjab,India,India,India,NaN
5,Haryana,Bilari,Moradabad,UTTAR PRADESH,NaN
6,Haryana,Amroha,Jyotiba Phule Nagar,UTTAR PRADESH,NaN
7,Haryana,Hasanpur,Jyotiba Phule Nagar,UTTAR PRADESH,NaN
8,Haryana,India,India,India,NaN
10,Jind,Jind,Jind,HARYANA,NaN
12,Jind,India,India,India,NaN
14,Chandigarh,Thakurpukur Mahestola,South Twenty Four Parganas,WEST BENGAL,NaN
15,Chandigarh,Ramgarh,Alwar,RAJASTHAN,NaN


In [10]:
test_data

'Jan Lokpal Bill: Protests continue in support of Anna Hazare in Punjab  CHANDIGARH: Khap panchayats today joined the protests in support of Anna Hazare in Punjab and Haryana where the Gandhian s supporters took out car and bike rallies and staged a demonstration outside Prime Minister Manmohan Singh s private residence here.   As the protests spread, people in rural areas were seen participating in demonstrations in support of Hazare.   Khap spokesman Suresh Koth said the khap panchayats (caste councils) in Haryana are observing a hunger strike in Jind, the Jat heartland of Haryana, and other places in support of Hazare s movement.   In Chandigarh, activists of various organisations staged a protest outside the Prime Minister s private residence in Sector 11.   The Chandigarh administration had yesterday promulgated prohibitory orders banning assembly of five or more persons outside the residences of five VIPs, including that of the Prime Minister.   The Additional District Magistrate